In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
from sqlite3 import connect

In [45]:
with zipfile.ZipFile("da_test_v2.zip","r") as z :
    z.extractall("da_test_v2")

In [46]:
df_game_session = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\gamesession.csv")
df_clips = pd.read_csv("da_test_v2/da_test_v2/clips.csv")
df__downloaded_clips = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\downloaded_clips.csv")
df_premium = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\premium.csv")
df_shared_clips = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\shared_clips.csv")

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\A'
C:\Users\aerif\AppData\Local\Temp\ipykernel_26788\1901619246.py:1: SyntaxWarning: invalid escape sequence '\A'
  df_game_session = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\gamesession.csv")
C:\Users\aerif\AppData\Local\Temp\ipykernel_26788\1901619246.py:3: SyntaxWarning: invalid escape sequence '\A'
  df__downloaded_clips = pd.read_csv("E:\AI Data Optimization\da_test_v2\da_test_v2\downloaded_clips.csv")
C:\Users\aerif\AppData\Local\Temp\ipykernel_26788\1901619246.py:4: SyntaxWarning: invalid escape sequence '\A'
  df_premium = pd.read_csv("E:\AI Data Optimization\da_test_v2\

In [47]:
conn = connect(':memory:')
df_game_session.to_sql("game_session", conn, index=False, if_exists="replace")
df_clips.to_sql("clips", conn, index=False, if_exists="replace")
df__downloaded_clips.to_sql("downloaded_clips", conn, index=False, if_exists="replace")
df_premium.to_sql("premium", conn, index=False, if_exists="replace")
df_shared_clips.to_sql("shared_clips", conn, index=False, if_exists="replace")

5798

In [ ]:
q1 = """
SELECT COUNT(DISTINCT gs.user_id) AS active_streamers
FROM game_session gs
JOIN clips c ON gs.id = c.gamesession_Id;
"""
df_q1 = pd.read_sql(q1, conn)
print("Active Streamers:\n", df_q1, "\n")

Active Streamers:
    active_streamers
0              4180 



In [ ]:
q2 ="""
SELECT gs.user_id,
       COUNT(c.id) AS total_clips
FROM game_session as gs
JOIN clips c ON gs.id = c.gamesession_Id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
"""
df_q2 = pd.read_sql(q2, conn)
print("Top 10 User Productive :\n", df_q2)

Top 10 User Productive :
    user_id  total_clips
0  3786671         5997
1  3343826         5545
2  2989826         4545
3  3460439         4514
4  3489200         3852
5   786188         3553
6  3911186         3405
7  4347273         3305
8  3033978         3124
9  3558278         3064


In [50]:
q3 = """
SELECT 
    c.id AS clip_id,
    COUNT(DISTINCT d.id) AS downloads,
    COUNT(s.clip_id) AS shares
FROM clips c
LEFT JOIN downloaded_clips d ON c.id = d.clip_id
LEFT JOIN shared_clips s ON c.id = s.clip_id
GROUP BY c.id
ORDER BY 2 DESC, 3 DESC
LIMIT 10;
"""
df_q3 = pd.read_sql(q3, conn)
print("Clip Engagement:\n", df_q3)

Clip Engagement:
         clip_id  downloads  shares
0  143709813465          2       0
1  182774099705          2       0
2  187204544695          2       0
3  190533788445          1       6
4  137028007670          1       5
5  189831754795          1       5
6  190481141745          1       5
7  191090826250          1       5
8  136464726805          1       4
9  161703492665          1       4


In [53]:
q4 = """
SELECT gs.game_name,
       COUNT(c.id) AS total_clips,
       SUM (gs.duration) AS total_duration
FROM game_session gs
JOIN clips c ON gs.id = c.gamesession_id
GROUP BY gs.game_name
ORDER BY total_clips DESC
LIMIT 10;
"""
df_q4 = pd.read_sql(q4, conn)
print("Top Games :\n",df_q4)

Top Games :
                          game_name  total_clips  total_duration
0          COD: Warzone3-2 & MW3-2       139790   2875443516324
1                      Other Games       130047   2687975000212
2                         Fortnite        65718   1377666747246
3                            SMITE        48761    986355423413
4                     Apex Legends        29234    605685865994
5                League of Legends        24759    512615742060
6                World of Warcraft        22004    437121344777
7                    Rocket League        11993    237043302636
8                          Podcast        10579    217181407773
9  Tom Clancy's Rainbow Six: Siege         7587    153513031733


In [55]:
q5 = """
SELECT 
    CASE WHEN p.user_id IS NOT NULL THEN 'Premium' ELSE 'Free' END AS user_type,
    AVG(clip_count) AS avg_clips_per_user
FROM (
    SELECT gs.user_id, COUNT(c.id) AS clip_count
    FROM game_session gs
    JOIN clips c ON gs.id = c.gamesession_id
    GROUP BY gs.user_id
) sub
LEFT JOIN premium p ON sub.user_id = p.user_id
GROUP BY user_type;
"""
df_q5 = pd.read_sql(q5, conn)
print("Premium User VS Free User :\n", df_q5)

Premium User VS Free User :
   user_type  avg_clips_per_user
0      Free          128.540541
1   Premium          131.764706
